# 2.4 优化RAG应用提升问答准确度
## 🚄  前言

在前面的课程中，你已经通过自动化评测发现答疑机器人的一些问题。优化提示词并不能解决检索召回不准确引起的回答错误问题，就像你在开卷考试时拿着错误的参考书很难给出正确的答案一样。  

这一章节，我们将一起更深入地了解 RAG 工作流程，并尝试提升我们的 RAG 应用问答准确度。


## 🍁 课程目标
学完本课程后，你将：

*   更深入的了解 RAG 的实现原理与技术细节
    
*   了解 RAG 应用常见的问题与处理方式建议
    
*   通过案例，动手改进 RAG 应用效果

## 1. 前文回顾

在上一章节，你发现答疑机器人不能很好的回答「张伟是哪个部门的？」，并且发现了原因是检索阶段没有召回到正确的参考信息（文档切片）。

In [16]:
import os
from config.load_key import load_key
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


In [31]:
from chatbot import rag

query_engine = rag.create_query_engine(rag.load_index())

def ask(question, query_engine):
    rag.update_prompt_template(query_engine=query_engine)

    print('提问：' + question)
    response = query_engine.query(question)

    print('回答：', end='')
    response.print_response_stream()

    print('\n\n检索召回的文档切片如下：')
    for source_node in response.source_nodes:
        print(source_node)

    return response

response = ask('张伟是哪个部门的', query_engine)

提问：张伟是哪个部门的
回答：根据提供的参考信息，没有找到名为张伟的员工信息。如果您能提供更多详细信息，例如部门或其他联系方式，我可以帮助您进一步查找。

检索召回的参考信息如下：
Node ID: c98bf1c0-c88d-43ee-b3ef-56079b335404
Text: 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。  ⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034
qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，
Score:  0.171

Node ID: daa30e2b-cbb1-4a5a-b2ae-e7f5cae4e3b9
Text: ⽀持。  绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041
hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效
Score:  0.169



## 2. 初步优化检索效果

### 2.1 让大模型获取到更多参考信息
我们可以尝试调整检索策略，比如一次多检索出几个文档切片。

#### 2.1.1 调整代码

In [42]:
from llama_index.llms.dashscope import DashScope

index = rag.load_index()
query_engine = index.as_query_engine(
    streaming=True,
    llm=DashScope(model_name="qwen-plus"),
    # 一次检索出 5 个文档切片，默认为 2
    similarity_top_k=5
)

response = ask('张伟是哪个部门的', query_engine)

提问：张伟是哪个部门的
回答：张伟是IT部的IT专员。

检索召回的参考信息如下：
Node ID: c98bf1c0-c88d-43ee-b3ef-56079b335404
Text: 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。  ⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034
qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，
Score:  0.171

Node ID: daa30e2b-cbb1-4a5a-b2ae-e7f5cae4e3b9
Text: ⽀持。  绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041
hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效
Score:  0.169

Node ID: 445d424e-d896-4bde-b4de-85f7a4e78ee4
Text: 效管理部 南 ⻜ 0 ⼒资源 效专员 0
责制定绩效考核体系，组织绩效评估的实施与反馈，撰写评估报告，分析绩效数据以提出优化建议，提供决策
Score:  0.154

Node ID: 5d98660f-2745-400a-bf45-eca616189317
Text: 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。  IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员
13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置
Score:  0.140



可以看到，调整了召回数量后，你的答疑机器人能够回答「张伟是哪个部门？」这个问题了。

不过，事实上你们公司名为张伟的同事有多个，他们分别在教研部、课程开发部、IT部。因此，这个答案还是不够好。

#### 2.1.2 评估改进效果
为了在接下来的改进中，能够量化改进效果，我们可以继续使用上一章节中的 ragas 来进行评测。

In [41]:
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings import DashScopeEmbeddings
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_recall,context_precision,answer_correctness

def evaluate_result(question, response, ground_truth):
    answer = response.response_txt
    context = [source_node.get_content() for source_node in response.source_nodes]

    data_samples = {
        'question': [question],
        'answer': [answer],
        'ground_truth':[ground_truth],
        'contexts' : [context],
    }
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(
        dataset = dataset,
        metrics=[answer_correctness, context_recall, context_precision],
        llm=Tongyi(model_name="qwen-plus"),
        embeddings=DashScopeEmbeddings(model="text-embedding-v3")
    )
    return score.to_pandas()

question = '张伟是哪个部门的'
ground_truth = '''公司有三名张伟，分别是：
- 教研部的张伟：职位是教研专员，邮箱 zhangwei@educompany.com。
- 课程开发部的张伟：职位是课程开发专员，邮箱 zhangwei01@educompany.com。
- IT部的张伟：职位是IT专员，邮箱 zhangwei036@educompany.com。
'''

In [43]:
evaluate_result(question=question, response=response, ground_truth=ground_truth)

Evaluating: 100%|██████████| 3/3 [00:38<00:00, 12.89s/it]


,question,answer,ground_truth,contexts,answer_correctness,context_recall,context_precision
0,张伟是哪个部门的,张伟是IT部的IT专员。,公司有三名张伟，分别是：\n- 教研部的张伟：职位是教研专员，邮箱 zhangwei@edu...,[核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 \n⾏政部 秦⻜ 蔡静 G705 034 ...,0.364468,0.25,0.25


### 2.2 给大模型结构更清晰的参考信息

在前面的改进过程中，你可能已经发现，这些参考信息原本是表格里的，但是在建立索引的过程中，被处理成了不太有结构的文本。

结构清晰的文本，对于大模型来说是更好的输入。我们在对原始文档切片、使用向量模型建立索引的过程中，如果输入的文档切片也是结构清晰的，同样有利于改善检索效果。在 RAG 场景下，markdown 结构简单且清晰，更有利检索和模型参考总结。

#### 2.2.1 重建索引
为了验证这一点，你可以尝试执行下面的命令，将我们预先准备好的 markdown 文件添加到 docs 目录中，然后重新建立索引。


In [44]:
# 将 markdown 格式的员工信息文档复制到 ./docs 目录下
! cp ./resources/2_4/内容公司各部门职责与关键角色联系信息汇总.md ./docs/

In [45]:
from llama_index.llms.dashscope import DashScope
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels

# 重新建立索引
print('重建索引')
documents = SimpleDirectoryReader('./docs').load_data()
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=DashScopeEmbedding(model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2)
)

query_engine = index.as_query_engine(
    streaming=True,
    llm=DashScope(model_name="qwen-plus"),
    similarity_top_k=5
)

response = ask('张伟是哪个部门的', query_engine)

重建索引
提问：张伟是哪个部门的
回答：张伟分别在三个不同的部门任职：

1. 教研部 - 内容设计教研专员，工号001，邮箱zhangwei@educompany.com
2. 课程开发部 - 内容开发课程开发专员，工号008，邮箱zhangwei01@educompany.com
3. IT部 - IT支撑IT专员，工号036，邮箱zhangwei036@educompany.com

请问您需要了解哪一个张伟的信息？

检索召回的参考信息如下：
Node ID: 2bb789a1-b549-4c26-9d1f-70ba4d2f3fc2
Text: 各部门关键角色联系人  | 部门|员工姓名|员工主管|工位|工号|岗位|职位|电话|邮箱|工作职责| |
---|---|---|---|---|---|---|---|---|---| | 教研部|张伟|李琳|A101|001|内容设计|教研专
员|13800000001|zhangwei@educompany.com|负责教育课程的研究与开发,分析教学效果,整理教案,协助课程优化,
以及参与教育项目的评估和反馈。| | 教研部|王芳|李琳|A102|002|内容设计|教研专员|13800000002|wangfang@e
ducompany.com|负责制定学科教学方案,策划教学活动,编写教案,收集学生反馈,参与课程改进会议,提供专业意见。| |
教研部|刘杰|李琳|A103|003|内...
Score:  0.241

Node ID: 56f642fb-4bdb-4415-ae96-360379329802
Text: com|进行公司网络及硬件设备的配置与维护,监控系统运行状态,及时处理技术问题与故障,提供技术支持及工具使用培训。| | IT部
|谢宇|马云|H803|037|IT支撑|IT专员|13800000037|xieyu@educompany.com|支持公司软件系统的开发
与更新,参与IT项目的计划与实施,撰写技术文档与使用说明,确保信息技术的安全性与有效性。| |
绩|朱|李|I901|04|人|绩|1380000004|zhun@educompany.com|负| | 效管理部|南|飞||0|力资源|
效专员|0||责制定绩效考核体系,组织绩效评估的实施与反馈,撰写评估报告,分析绩效数据以

#### 2.2.2 评估改进效果
可以看到，你的答疑机器人能够准确回答这个问题。你可以再运行一次 ragas 评测，评测数据同样显示，回答准确度更高了。

In [46]:
evaluate_result(question=question, response=response, ground_truth=ground_truth)

Evaluating: 100%|██████████| 3/3 [01:01<00:00, 20.38s/it]


,question,answer,ground_truth,contexts,answer_correctness,context_recall,context_precision
0,张伟是哪个部门的,张伟分别在三个不同的部门任职：\n\n1. 教研部 - 内容设计教研专员，工号001，邮箱z...,公司有三名张伟，分别是：\n- 教研部的张伟：职位是教研专员，邮箱 zhangwei@edu...,[各部门关键角色联系人\n\n| 部门|员工姓名|员工主管|工位|工号|岗位|职位|电话|邮...,0.788695,1.0,0.833333


## 3. 了解一个简单 RAG 应用的工作流程

截止目前，你已经完成了一些改进，让答疑机器人的问答准确度更高了。但在实际生产环境中，你可能会遇到的问题远不止于此。

系统全面地了解 RAG 工作流程，有助于你进一步提升回答效果。

RAG 全称 Retrieval Augmented Generation，即检索增强生成，它的主要工作流程也正如其名：

1. **建立索引**：对文档建立索引，为后续检索做准备。索引阶段又可以分为：文档解析与切片、切片向量存储。。
2. **检索与生成**：根据用户的问题，从向量数据库中检索出与用户输入最相关的文档片段，然后输入给大模型，生成答案。
   
   
<img src="https://img.alicdn.com/imgextra/i4/O1CN018d8e9G1V0jDAZMRXp_!!6000000002591-0-tps-1463-997.jpg" alt="RAG 工作原理" width="800px">

我们在前面的学习中对答疑机器人做出了一些改进。但 RAG 应用的改进点远不止于此，每个环节都有改进空间。

## 4. 简单 RAG 应用的常见问题及改进策略

### 4.1 文档解析与切片阶段

大模型在回答问题时拿到的文档切片如果缺少关键信息，会回答不准确；如果拿到的文档切片非关联信息过多（噪声），也会影响回答质量。信息过少或过多，同样也会影响检索效果。

因此，在建立索引的前期，对文档进行解析与切片阶段时，需要想办法最终的切片信息要完整，但也不要包含太多干扰信息。


#### 4.1.1 问题分类及改进策略
在文档解析与切片阶段，从参考信息完整度的角度，我们可以把可能出现的问题分为两大类：参考信息缺失、干扰信息过多。

<table>
  <tr>
    <th>问题大类</th>
    <th>细分类型</th>
    <th>改进策略</th>
  </tr>
  <tr>
    <td rowspan="6">参考信息缺失</td>
    <td>没有相关知识文档</td>
    <td>补充文档</td>
  </tr>  
  <tr>  
    <td>有相关知识文档，但和用户问题表述方式不接近</td>
    <td>改进文档，补充用户视角描述信息</td>
  </tr> 
  <tr>    
    <td>文档类型不统一，部分格式的文档不支持解析<br><i style="color: #999">比如前面用到的 SimpleDirectoryLoader 并不支持 keynote 格式的文件<i></td>
    <td>开发对应格式的解析器，或转换文档格式</td>
  </tr>  
  <tr>    
    <td>已支持解析的文档格式里，存在一些特殊内容<br><i style="color: #999">比如文档里嵌入了表格、图片、视频等<i></td>
    <td>改进文档解析器</td>
  </tr>
  <tr>    
    <td>文档切片长度过短，有效信息被截断</td>
    <td>扩大切片长度，或结合具体业务开发为更合适的切片策略</td>
  </tr>  
  <tr>    
    <td>...</td>
    <td>...</td>
  </tr> 
  <tr>
    <td rowspan="3">干扰信息过多</td>
    <td>文档中有很多主题接近的内容<br><i style="color: #999">比如工作手册文档中，需求分析、开发、发布等每个阶段都有注意事项、操作指导<i></td>
    <td>扩写文档标题及子标题<br><i style="color: #999">「注意事项」=>「需求分析>注意事项」</i><br>建立文档元数据（打标）<br>...</td>
  </tr>  
  <tr>    
    <td>文档切片长度过大，引入过多干扰项</td>
    <td>减少切片长度，或结合具体业务开发为更合适的切片策略</td>
  </tr> 
   <tr>    
    <td>...</td>
    <td>...</td>
  </tr> 
</table>

#### 4.1.2 改进策略实现参考 - 借助百炼解析 PDF 文件

在前面的学习过程中，为了让你更快地看到格式转换带来的效果，我们直接提供了一份从 PDF 转换的 Markdown 格式文档。但在实际工作中，编写代码将 PDF 妥善地转为 Markdown 并非易事。

实际工作中，你也可以借助百炼提供的 DashScopeParse 来完成 PDF、Word 等格式的文件解析。DashScopeParse 背后使用了阿里云的[文档智能](https://www.aliyun.com/product/ai/docmind)服务，能够帮助你从 PDF、Word 等格式的文件中识别文档中的图片、提取出结构化的文本信息。

In [10]:
from llama_index.readers.dashscope.utils import ResultType
from llama_index.readers.dashscope.base import DashScopeParse

import os
import re
import json
import nest_asyncio

nest_asyncio.apply()
# 使用环境变量
os.environ['DASHSCOPE_API_KEY'] = os.getenv('DASHSCOPE_API_KEY')

# 文件通过DashScopeParse接口解析为程序与大模型易于处理的markdown文本
def file_to_md(file, category_id):
    parse = DashScopeParse(
        result_type=ResultType.DASHSCOPE_DOCMIND,
        category_id=category_id
    )
    documents = parse.load_data(file_path=file)
    # 初始化一个空字符串来存储Markdown内容
    markdown_content = ""
    for doc in documents:
        doc_json = json.loads(json.loads(doc.text))
        for item in doc_json["layouts"]:
            if item["text"] in item["markdownContent"]:
                markdown_content += item["markdownContent"]
            else:
                # DashScopeParse处理时，会将文档图片内的文本信息也解析到初始markdown文本中（类似OCR），这对于本文示例文件中的命令行截图、文本截图是足够的，示例无需深层次解析图片。
                # 实际场景中的知识库文档，如果涉及不规则、复杂信息的图片并且需要深层次理解图片内容，您可以调用视觉模型进一步理解图片含义。
                # （DashScopeParse返回的数据结构中，针对图片数据，markdownContent字段是图片url，text字段是解析出的文本）
                # if ".jpg" in item["markdownContent"] or ".jpeg" in item["markdownContent"] or ".png" in item["markdownContent"]:
                #     image_url = re.findall(r'\!\[.*?\]\((https?://.*?)\)', item["markdownContent"])[0]
                #     print(image_url)
                #     markdown_content = markdown_content + parse_image_to_text(image_url)+"\n"
                # else:
                #     markdown_content = markdown_content + item["text"]+"\n"
                markdown_content = markdown_content + item["text"]+"\n"
    return markdown_content

### 调用示例

# 1、可选配置。
# 百炼平台上，可以对不同项目配置不同的业务空间，默认情况下是使用默认业务空间。
# 如果需要使用非默认空间，可以前往[百炼控制台-业务空间管理](https://bailian.console.aliyun.com/?admin=1#/efm/business_management)，配置业务空间并获取Workspace ID。
# 完成后，取消注释并修改这段代码为实际值：
# os.environ['DASHSCOPE_WORKSPACE_ID'] = "<Your Workspace id, Default workspace is empty.>"

# 2、可选配置。
# 文件通过DashScopeParse进行解析时，需要配置上传的数据目录id。可以前往[百炼控制台-数据管理](https://bailian.console.aliyun.com/#/data-center)，配置类目并获取ID
category_id="default" # 建议修改为自定义的类目ID，以便分类管理文件

md_content = file_to_md(['./docs/内容公司各部门职责与关键角色联系信息汇总.pdf'], category_id)
print(md_content)

Parsing files:   0%|          | 0/1 [00:00<?, ?it/s]2024-11-04 13:01:51,866 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/api/v1/datacenter/category/default/upload_lease "HTTP/1.1 200 OK"
2024-11-04 13:01:52,041 DashScopeResponseHandler [INFO] 8471001792 : DashScopeParse upload_lease [URL:https://dashscope.aliyuncs.com/api/v1/datacenter/category/default/upload_lease] request_id: 5693ab02-00f0-936d-89a8-cb8042e82f8b.
2024-11-04 13:01:52,041 - INFO - DashScopeParse upload_lease [URL:https://dashscope.aliyuncs.com/api/v1/datacenter/category/default/upload_lease] request_id: 5693ab02-00f0-936d-89a8-cb8042e82f8b.
2024-11-04 13:01:52,042 llama_index.readers.dashscope.base [INFO] 8471001792 : ./docs/内容公司各部门职责与关键角色联系信息汇总.pdf upload lease result: bfd73c9058d946cfa9c7cf49e4d20d0c.1730696511978
2024-11-04 13:01:52,042 - INFO - ./docs/内容公司各部门职责与关键角色联系信息汇总.pdf upload lease result: bfd73c9058d946cfa9c7cf49e4d20d0c.1730696511978
2024-11-04 13:01:52,044 llama_index.readers.dashscope.domai

# 内容公司各部门职责与关键角色联系方式  

## 公司定位  

我们的公司致力于推动教育的数字化转型,专注于开发富有创意和互动性的在线学习平台与工具。我们结合最新的教育理论与技术创新,旨在提升学习者的参与度和学习效果。通过提供个性化学习体验,我们希望满足不同年龄段和背景的学习需求,助力教育公平和终身学习。我们的使命是使知识获取更加便捷、高效,让每个人都能充分发挥潜力。  

## 各部门介绍  

教研部:教育课程研究设计  

课程开发部:技术内容需求开发  

教材编写部:教材、练习题等内容汇编与修订  

评估部:内容质量质检  

市场部:市场活动营销  

人力资源部:人力资源管理  

IT部:IT技术支撑  

绩效管理部:人员绩效考核设计  

## 各部门关键角色联系人  

| 部门|员工姓名|员工主管|工位|工号|岗位|职位|电话|邮箱|工作职责|
| ---|---|---|---|---|---|---|---|---|---|
| 教研部|张伟|李琳|A101|001|内容设计|教研专员|13800000001|zhangwei@educompany.com|负责教育课程的研究与开发,分|
| |||||||||析	教学效果,整理教案,协助课程优化,以及参与教育项目的评估和反<br>馈。|
| 教研部|王芳|李琳|A102|002|内容设计|教研专员|13800000002|wangfang@educompany.com|负责制定学科教学方案,策划教学|
| |||||||||活动,编写教案,收集学生反馈,参与课程改进会议,提供专业意<br>见。|
| 教研部|刘杰|李琳|A103|003|内容设计|教研专员|13800000003|liujie@educompany.com|组织教研活动,分析教学成果,开展学术研|
| |||||||||究,协助撰与研究报告,输出有效教学策略,提升课程品质。|
| 课程开发部|李丽|王强|B201|007|内容开发|课程开发专员|13800000007|lili@educompany.com|设计并开发教育课程,撰写课程大纲,组织课程<br>试|
| |||||||||点,分析市场需求,调整课程内容,以适应学习者的需求和反馈。|
| 课程开发部|张伟|王强|B202|008|内容开分发|课程开发专

由于pdf/docx等多种文件格式来源的多样性，文件解析到markdown过程中可能存在一些格式上的小问题，比如 PDF 里的跨页表格行可能被解析为多行。

可以使用大模型对生成的markdown文本进行润色，修正目录层级、缺失信息等。

In [14]:
from dashscope import Generation

def md_polisher(data):
    messages = [
        {'role': 'user', 'content': '下面这段文本是由pdf转为markdown的，格式和内容可能存在一些问题，需要你帮我优化下：\n1、目录层级，如果目录层级顺序不对请以markdown形式补全或修改；\n2、内容错误，如果存在上下文不一致的情况，请你修改下；\n3、如果有表格，注意上下行不一致的情况；\n4、输出文本整体应该与输入没有较大差异，不要自己制造内容，我是需要对原文进行润色；\n4、输出格式要求：markdown文本，你的所有回答都应该放在一个markdown文件里面。\n特别注意：只输出转换后的 markdown 内容本身，不输出任何其他信息。\n需要处理的内容是：' + data}
    ]
    response = Generation.call(
        model="qwen-plus",
        messages=messages,
        result_format='message',
        stream=True,
        incremental_output=True
    )
    result = ""
    for chunk in response:
        print(chunk.output.choices[0].message.content, end='')
        result += chunk.output.choices[0].message.content

    return(result)

md_polisher(md_content)

# 内容公司各部门职责与关键角色联系方式

## 公司定位

我们的公司致力于推动教育的数字化转型，专注于开发富有创意和互动性的在线学习平台与工具。我们结合最新的教育理论与技术创新，旨在提升学习者的参与度和学习效果。通过提供个性化学习体验，我们希望满足不同年龄段和背景的学习需求，助力教育公平和终身学习。我们的使命是使知识获取更加便捷、高效，让每个人都能充分发挥潜力。

## 各部门介绍

- **教研部**: 教育课程研究设计
- **课程开发部**: 技术内容需求开发
- **教材编写部**: 教材、练习题等内容汇编与修订
- **评估部**: 内容质量质检
- **市场部**: 市场活动营销
- **人力资源部**: 人力资源管理
- **IT部**: IT技术支撑
- **绩效管理部**: 人员绩效考核设计

## 各部门关键角色联系人

| 部门       | 员工姓名 | 员工主管 | 工位  | 工号 | 岗位       | 职位           | 电话          | 邮箱                        | 工作职责                                                                                   |
|------------|----------|----------|-------|------|------------|----------------|---------------|-----------------------------|--------------------------------------------------------------------------------------------|
| 教研部     | 张伟     | 李琳     | A101  | 001  | 内容设计   | 教研专员       | 13800000001  | zhangwei@educompany.com     | 负责教育课程的研究与开发，分析教学效果，整理教案，协助课程优化，以及参与教育项目的评估和反馈。 |
| 教研部     | 王芳     | 李琳     | A102  | 002  |

'# 内容公司各部门职责与关键角色联系方式\n\n## 公司定位\n\n我们的公司致力于推动教育的数字化转型，专注于开发富有创意和互动性的在线学习平台与工具。我们结合最新的教育理论与技术创新，旨在提升学习者的参与度和学习效果。通过提供个性化学习体验，我们希望满足不同年龄段和背景的学习需求，助力教育公平和终身学习。我们的使命是使知识获取更加便捷、高效，让每个人都能充分发挥潜力。\n\n## 各部门介绍\n\n- **教研部**: 教育课程研究设计\n- **课程开发部**: 技术内容需求开发\n- **教材编写部**: 教材、练习题等内容汇编与修订\n- **评估部**: 内容质量质检\n- **市场部**: 市场活动营销\n- **人力资源部**: 人力资源管理\n- **IT部**: IT技术支撑\n- **绩效管理部**: 人员绩效考核设计\n\n## 各部门关键角色联系人\n\n| 部门       | 员工姓名 | 员工主管 | 工位  | 工号 | 岗位       | 职位           | 电话          | 邮箱                        | 工作职责                                                                                   |\n|------------|----------|----------|-------|------|------------|----------------|---------------|-----------------------------|--------------------------------------------------------------------------------------------|\n| 教研部     | 张伟     | 李琳     | A101  | 001  | 内容设计   | 教研专员       | 13800000001  | zhangwei@educompany.com     | 负责教育课程的研究与开发，分析教学效果，整理教案，协助课程优化，以及参与教育项目的评估和反馈。 |\n| 教研部     | 王芳     | 

通过上面的步骤，你已经成功地将 PDF 转成了 Markdown，并且修正了其中的格式问题。与此同时，即使文档中存在图片，图片中的信息也能被提取出来，以便构建更有利于检索效果的知识库。

### 4.2 切片向量化与存储阶段

文档切片后，我们还需要对其建立索引，以便后续检索。一个常见的方案是使用嵌入（Embedding）模型将切片向量化，并存储到向量数据库中。

在这一阶段，你需要选择合适的 Embedding 模型以及向量数据库，这对于提升检索效果至关重要。

#### 4.2.1 了解 Embedding 与向量化
Embedding 模型可以将文本转换为高维向量，用于表示文本语义，相似的文本会映射到相近的向量上，检索时可以根据问题的向量找到相似度高的文档切片。
<style>
  table.no-border,
  table.no-border td {
    border: none;
    vertical-align: top;
  }  
  .note {
    color: #999;
    font-size: 0.8em;
  }  
</style>
<div class="note">
  <p>
    <i>平面坐标系中的有向线段是 2 维向量。例如，从原点 (0, 0) 到 A (xa, ya) 的有向线段可以称为向量 A。向量 A 与向量 B 之间的夹角越小，也就意味着其相似度越高。</i>  
  </p>  
  <img src="https://img.alicdn.com/imgextra/i2/O1CN01DkQ8nK1Z60EgwJq9S_!!6000000003144-0-tps-1556-1382.jpg" width="200" style="opacity: 0.4"></td>
</div>

In [48]:
import numpy as np

def cosine_similarity(a, b):
    """余弦相似度"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# 示例向量
a = np.array([0.2, 0.8])
b = np.array([0.3, 0.7])
c = np.array([0.8, 0.2])

print(f"A 与 B 的余弦相似度: {cosine_similarity(a, b)}")
print(f"B 与 C 的余弦相似度: {cosine_similarity(b, c)}")

A 与 B 的余弦相似度: 0.987241120712647
B 与 C 的余弦相似度: 0.6050832675335579


#### 4.2.2 选择合适的 Embedding 模型
不同的 Embedding 模型对相同的几段文字进行计算时，得到的向量可能会完全不同。通常越新的 Embedding 模型，其表现越好。例如前文中使用的是阿里云百炼上提供的 text-embedding-v2。如果你换成更新的版本 [text-embedding-v3](https://help.aliyun.com/zh/model-studio/user-guide/embedding)，你会发现即使不去做前面的优化，检索效果也会有一定的提升。

比如运行下面的代码可以看到，不同版本的 Embedding 模型，对于「张伟是哪个部门的」这个问题和不同文档切片的相似度也是不同的。

In [47]:
from llama_index.embeddings.dashscope import DashScopeEmbedding

query = "张伟是哪个部门的"
chunk_1 = "核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 ⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布"
chunk_2 = "组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置"

text_embedding_v2 = DashScopeEmbedding(model_name="text-embedding-v2")
text_embedding_v3 = DashScopeEmbedding(model_name="text-embedding-v3")

print(f"query: {query}")
print(f"chunk_1: {chunk_1}")
print(f"chunk_2: {chunk_2}")

print("\n=== text-embedding-v2 ===")
print(f"""query 和 chunk_1 的相似度: {cosine_similarity(text_embedding_v2.get_text_embedding(query), text_embedding_v2.get_text_embedding(chunk_1))}""")
print(f"""query 和 chunk_2 的相似度: {cosine_similarity(text_embedding_v2.get_text_embedding(query), text_embedding_v2.get_text_embedding(chunk_2))}""")

print("=== text-embedding-v3 ===")
print(f"""query 和 chunk_1 的相似度: {cosine_similarity(text_embedding_v3.get_query_embedding(query), text_embedding_v3.get_text_embedding(chunk_1))}""")
print(f"""query 和 chunk_2 的相似度: {cosine_similarity(text_embedding_v3.get_query_embedding(query), text_embedding_v3.get_text_embedding(chunk_2))}""")

query: 张伟是哪个部门的
chunk_1: 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 ⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布
chunk_2: 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置

=== text-embedding-v2 ===
query 和 chunk_1 的相似度: 0.3689517525912472
query 和 chunk_2 的相似度: 0.25767606999575354
=== text-embedding-v3 ===
query 和 chunk_1 的相似度: 0.49452193642238024
query 和 chunk_2 的相似度: 0.5603671453781736


另外，通用的 Embedding 模型是基于通用领域语料训练，可能并不适合某些细分垂直领域，如医学、金融、法律。因为这些领域有特定的术语和复杂知识结构。为解决这一问题，通常需要在专业数据上训练专门的 Embedding 模型，或者对通用模型进行微调，以提高其在特定领域的表现。

#### 4.2.3 选择合适的向量数据库

文档切片转为向量后，需要持久化存储下来，用于后续检索。目前构建的答疑机器人中，使用的是 LlamaIndex 内置的内存向量存储。实际生产环境中，选择一个合适的向量数据库是更好的选择。

很多向量数据库除了向量相似度检索外，还支持基于元数据标签过滤（也称标量检索）。如果你的知识库里有很多相似的内容（比如前面的岗位说明里有，每个岗位都有注意事项），就可以结合标量检索进一步缩小检索范围，让检索更准确。

阿里云上有很多云服务提供了向量存储与检索能力，你可以结合具体业务需求、既有资源情况以及人员能力来选择合适的向量数据库：

1. 如果你是第一次构建 RAG 应用，并且希望控制初期投入成本，你可以选择 [向量检索服务（DashVector）](https://www.aliyun.com/product/ai/dashvector) 。DashVecctor支持按实际调用量、存储量付费，能有效降低初期投入成本，并且能自动扩展以应对业务增长。
   
2. 如果已经在使用一些开源向量数据库，如 Milvus，则可以选择 [向量检索服务 Milvus 版](https://www.aliyun.com/product/milvus)。你可以方便的将数据迁移到阿里云上的向量检索服务 Milvus 版，并且无需关注 Milvus 数据的运维。
   
3. 如果你已经在使用阿里云上的数据库、搜索类产品，也可以使用这些产品所支持的向量存储与检索能力，无需购买额外资源：比如云数据库 RDS、云数据库 PolarDB、云原生多模数据库 Lindorm、云数据库 Redis、表格存储、智能开放搜索 OpenSearch、检索分析服务 Elasticsearch版 等都支持向量存储与检索。


<figure align="center">
  <img src="https://img.alicdn.com/imgextra/i4/O1CN01ked0xy1y2DM02aWzJ_!!6000000006520-0-tps-1942-932.jpg" width="600"/>
  <figcaption style="color: #999">DashVectory 中支持 age、name 等标签过滤 + 向量相似度检索</figcaption>
</figure>

### 4.3 检索召回阶段

检索阶段会遇到的主要问题就是，很难从众多文档切片中，找出和用户问题最相关、且包含正确答案信息的片段。

从切入时机来看，可以将解法分为两大类：

1. 在执行检索前，很多用户问题描述是不完整、甚至有歧义的，你需要想办法还原用户真实意图，以便提升检索效果。
2. 在执行检索后，你可能会发现会存在一些无关的信息，需要想办法减少无关信息，避免干扰下一步的答案生成。


<style>
  .note {
    color: #999;
    font-size: 0.9em;
  }
</style>
<table>
  <tr>
    <th>时机</th>
    <th>改进策略</th>  
    <th>示例</th>  
  </tr>
  <tr>
    <td rowspan="7">检索前</td>
    <td>问题改写<br><i class="note">通过重新表达，让问题更匹配数据库中的信息</i></td>
    <td>「附近有好吃的餐厅吗？」=> 「请推荐我附近的几家评价较高的餐厅」</td>
  </tr>
  <tr>    
    <td>问题扩写<br><i class="note">通过增加更多信息，让检索结果更全面</i></td>
    <td>「张伟是哪个部门的？」=> 「张伟是哪个部门的？他的联系方式、职责范围、工作目标是什么？」</td>
  </tr>
  <tr>    
    <td>基于用户画像扩展上下文<br><i class="note">结合用户信息、行为等数据扩写问题</i></td>
    <td><p>内容工程师提问「工作注意事项」=> 「内容工程师有哪些工作注意事项」</p><p>项目经理提问「工作注意事项」=> 「项目经理有哪些工作注意事项」</p></td>
  </tr>
  <tr>    
    <td>提取标签<br><i class="note">提取标签，用于后续标签过滤+向量相似度检索</i></td>
    <td>「内容工程师有哪些工作注意事项」=> <ul><li>标签过滤：{"岗位": "内容工程师"}</li><li>向量检索：「内容工程师有哪些工作注意事项」</li></ul></td>
  </tr>  
  <tr>    
    <td>反问用户</td>
    <td>「工作职责是什么」=> 大模型反问：「请问你想了解那个岗位的工作职责」  <br>
      <i class="note">实现反问的提示词可以参考：<a href="https://help.aliyun.com/zh/model-studio/use-cases/create-an-ai-shopping-assistant" target="blank">10分钟构建能主动提问的智能导购</a></i>
    </td>      
  </tr>  
  <tr>    
    <td>思考并规划多次检索</td>
    <td>
      「张伟不在，可以找谁」<br>
      => 大模型思考规划： <br> 
      => task_1：张伟的职责是什么, task_2：${task_1_result}职责的人有谁<br>
      => 按顺序执行多次检索
    </td>      
  </tr>  
  <tr>
    <td>...</td>
    <td>...</td>
  </tr>
  <tr>
    <td rowspan="7">检索后</td>
    <td>重排序 ReRank + 过滤<br><i class="note">多数向量数据库会考虑效率，牺牲一定精确度，召回的切片中可能有一些实际相关性不够高</i></td>
    <td>chunk1、chunk2...、chunk10 <br>=> chunk 2、chunk4、chunk5</td>
  </tr>
  <tr>
    <td>滑动窗口检索<br><i class="note">在检索到一个切片后，补充前后相邻的若干个切片，让信息更完整，因为相邻的切片语义可能是比较相关的。</i></td>
    <td><p>常见的实现是句子滑动窗口。</p>例如，原始文本：「<span class="note">我们将在下周举行一次会议。</span>会议的主题是公司未来的发展方向。<span class="note">希望大家都能积极参与并提出自己的看法。</span>」<br> <br> 检索到的句子：「会议的主题是公司未来的发展方向。」 <br><br> 补充相邻的句子后：「<span class="note">我们将在下周举行一次会议。</span>会议的主题是公司未来的发展方向。<span class="note">希望大家都能积极参与并提出自己的看法。</span>」
    <br> 
    </td>
  </tr>  
  <tr>
    <td>...</td>
    <td>...</td>
  </tr>
</table>

#### 4.3.1 改进策略参考实现 - 提取标签

在 RAG 应用开发过程中，你可以使用大模型来提取标签，以便提升后续检索效果。

比如，你可以使用百炼上提供的 qwen-turbo，其成本低、速度快，适用于标签提取这种推理复杂度不高的场景。

In [68]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.getenv("DASHSCOPE_API_KEY"), base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

system_message = """你是一个标签提取专家，请按照下面的要求，从用户问题中提取标签信息。
---
【目前支持的标签】
- 岗位名称
- 部门名称
---
【输出要求】
1. 请用 json 输出，如：[{"key": "部门名称", "value": "教研部"}]、[]
2. 识别不出来时，value 可以为 null
---
用户的问题如下：
"""

def extract_tags(question):
  completion = client.chat.completions.create(
      # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
      model="qwen-turbo",
      messages=[
          {'role': 'system', 'content': system_message},
          {'role': 'user', 'content': question}
      ],
      # 确保生成结果是 JSON 格式，以便后续流程解析使用
      response_format={"type": "json_object"}
  )
  return completion.choices[0].message.content

print(extract_tags('内容工程师的工作职责是什么'))
print(extract_tags('课程开发部的内容工程师的工作职责是什么'))
print(extract_tags('张伟是哪个部门的'))
print(extract_tags('张伟是谁'))

[{"key": "岗位名称", "value": "内容工程师"}]
[{"key": "部门名称", "value": "课程开发部"}, {"key": "岗位名称", "value": "内容工程师"}]
[{"key": "部门名称", "value": null}]
[]


当然，在实际使用时，还需要在建立索引阶段，将文档切片向量加上对应的标签存入向量数据库，才能在检索时使用向量 + 标量的联合检索。

#### 4.3.2 改进策略参考实现 - 重排序



我们可以删除前面构建的 markdown 文件，来复现本章节最开始回答不好「张伟是哪个部门的」的状态。

In [ ]:
!rm ./docs/内容公司各部门职责与关键角色联系信息汇总.md

删除文件后，可以执行下面的代码。可以看到，我们设置了从向量数据库中检索召回 3 条相关的文档切片。

从结果上来看，这 3 条切片，事实上不够相关，问答机器人无法正确回答「张伟是哪个部门的」。

In [50]:
from llama_index.llms.dashscope import DashScope
from chatbot import rag

index = rag.create_index('./docs')
query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus"),
    similarity_top_k=3,
    streaming=True,
)

response = ask("张伟是哪个部门的", query_engine=query_engine)

提问：张伟是哪个部门的
回答：根据提供的参考信息，没有找到名为张伟的员工信息。如果您能提供更多详细信息，例如部门名称或其他联系方式，我可以帮助您进一步查找。

检索召回的参考信息如下：
Node ID: 4165d466-ae2d-45da-9e2f-597b43528944
Text: 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。  ⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034
qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，
Score:  0.171

Node ID: 2fd56095-4c84-468d-8e79-f353aa99983f
Text: ⽀持。  绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041
hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效
Score:  0.169

Node ID: 58b7012c-d8c9-41ab-9554-fd0c5912ceca
Text: 效管理部 南 ⻜ 0 ⼒资源 效专员 0
责制定绩效考核体系，组织绩效评估的实施与反馈，撰写评估报告，分析绩效数据以提出优化建议，提供决策
Score:  0.154



In [51]:
evaluate_result(question, response, ground_truth)

Evaluating: 100%|██████████| 3/3 [00:41<00:00, 13.73s/it]


,question,answer,ground_truth,contexts,answer_correctness,context_recall,context_precision
0,张伟是哪个部门的,根据提供的参考信息，没有找到名为张伟的员工信息。如果您能提供更多详细信息，例如部门名称或其他...,公司有三名张伟，分别是：\n- 教研部的张伟：职位是教研专员，邮箱 zhangwei@edu...,[核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 \n⾏政部 秦⻜ 蔡静 G705 034 ...,0.186401,0.0,0.0


我们可以调整代码，先从向量数据库中检索召回 20 条文档切片，再借助阿里云百炼提供的[文本排序模型](https://help.aliyun.com/zh/model-studio/getting-started/models#eafbfdceb7n03)进行重排序，并且筛选出其中最相关的 3 条参考信息。

运行代码后你可以看到，同样是 3 条参考信息，这次大模型能够准确回答问题了。

In [52]:
from llama_index.postprocessor.dashscope_rerank import DashScopeRerank
from llama_index.core.postprocessor import SimilarityPostprocessor

query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus"),
    # 先设置一个较大的召回切片数量
    similarity_top_k=20,
    streaming=True,
    node_postprocessors=[
        # 在rerank 模型中选择你最终想召回的切片个数，重排模型选择通义实验室的gte-rerank模型
        DashScopeRerank(top_n=3, model="gte-rerank"),
        # 设置一个相似度阈值，低于该阈值的切片会被过滤掉
        SimilarityPostprocessor(similarity_cutoff=0.2)
    ]
)

response = ask("张伟是哪个部门的", query_engine=query_engine)

提问：张伟是哪个部门的
回答：公司中有三位名叫张伟的员工，分别隶属于不同的部门：

1. 教研部的张伟，岗位为内容设计教研专员。
2. 课程开发部的张伟，岗位为内容开发课程开发专员。
3. IT部的张伟，岗位为IT支撑IT专员。

请问您需要了解哪一位张伟的信息？

检索召回的参考信息如下：
Node ID: 9ebe53cb-6dc2-4384-8ef1-b95d2fba0d91
Text: 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。  IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员
13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置
Score:  0.638

Node ID: 487d986f-d9bd-46d6-b231-4c61a41a56d0
Text: 内容公司各部⻔职责与关键⻆⾊联系⽅式 公司定位
我们的公司致⼒于推动教育的数字化转型，专注于开发富有创意和互动性的在线学习平台与⼯具。 我们结合最新的教育理论与技术创新，
旨在提升学习者的参与度和学习效果。 通过提供个性化学习体验， 我们希望满⾜不同年龄段和背景的学习需求，
助⼒教育公平和终⾝学习。我们的使命是使知识获取更加便捷、⾼效，让每个⼈都能充分发挥潜⼒。 各部⻔介绍 教研部：教育课程研究设计
课程开发部：技术内容需求开发 教材编写部：教材、练习题等内容汇编与修订 评估部：内容质量质检 市场部：市场活动营销 ⼈⼒资源部：⼈⼒资源管理
IT部：IT技术⽀撑 绩效管理部：⼈员绩效考核设计 各部⻔关键⻆⾊联系⼈ 部⻔ 员⼯姓名 员⼯主管 ⼯位 ⼯号 岗位 职位 电话 邮箱
⼯作职责 教...
Score:  0.574

Node ID: 4d5ca6e1-0802-4202-8c36-bea2e6c961d2
Text: 点，分析市场需求，调整课程内容，以适应学习者的需求和反馈。  课程开发部 张伟 王强 B202 008 内容开发 课程开发专员
13800000008 zhangwei01@educompany.com 精细化课程设计，确保课程符合国家教育标
Score:  0.551



In [53]:
evaluate_result(question, response, ground_truth)

Evaluating: 100%|██████████| 3/3 [01:09<00:00, 23.12s/it]


,question,answer,ground_truth,contexts,answer_correctness,context_recall,context_precision
0,张伟是哪个部门的,公司中有三位名叫张伟的员工，分别隶属于不同的部门：\n\n1. 教研部的张伟，岗位为内容设计...,公司有三名张伟，分别是：\n- 教研部的张伟：职位是教研专员，邮箱 zhangwei@edu...,[组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 \nIT部 张伟 ⻢云 H...,0.68934,1.0,1.0


### 4.4 生成答案阶段


大模型最终生成的答案可能还是不及预期。你遇到的问题可能会有：

1. 没有检索到相关信息，大模型捏造答案。
2. 检索到了相关信息，但是大模型没有按照要求生成答案。
3. 检索到了相关信息，大模型也给出了答案，但是你希望 AI 给出更全面的答案。


为了解决这些问题，你可以从以下角度着手分析与解决：

1. 选择合适的大模型：
   
   1. 如果只是简单的信息查询总结，小参数量的模型足以满足需求，比如 [qwen-turbo](https://help.aliyun.com/zh/model-studio/getting-started/models#ff492e2c10lub)。
   
   2. 如果你希望答疑机器人能完成较为复杂的逻辑推理，建议选择参数量更大、推理能力更强的大模型，比如 [qwen-plus](https://help.aliyun.com/zh/model-studio/getting-started/models#bb0ffee88bwnk) 甚至是 [qwen-max](https://help.aliyun.com/zh/model-studio/getting-started/models#cf6cc4aa2aokf)。
   
   3. 如果你构建的 RAG 应用面向一些非通用领域，如法律领域，建议使用面向特定领域训练的模型，如[通义法睿](https://help.aliyun.com/zh/model-studio/getting-started/models#f0436273ef1xm)。

2. 充分优化提示词模板，比如：

    1. 明确要求：你可以在提示词对于捏造答案（幻觉）的情况，你可以通过提示词要求大模型：「如果所提供的信息不足以回答问题，请明确告知“根据现有信息，我无法回答这个问题。”切勿编造答案。」

    2. 使用分隔符：检索召回的文档切片如果随意混杂在提示词里，人也很难看清整个提示词的结构，大模型也会受到干扰。建议将提示词和检索切片明确的分开，以便大模型能够正确地理解你的意图。

    3. 动态提示词模板：不同问题的回答范式可能是不同的，你可以借助大模型识别问题类型，然后映射使用不同的提示词模板。比如有些问题，你希望大模型先输出整体框架，然后再输出细节；有些问题你可能希望大模型言简意赅的给出结论。

3. 调整大模型的参数，比如：

      1. 如果你希望大模型输出在相同的问题下，输出的内容尽可能相同，你可以在每次模型调用时传入相同的seed值。
      
      2. 如果你希望大模型在回答用户问题时不要总是用重复的句子，你可以适当调高 presence_penalty 值。

      3. 你也可以查阅[通义千问 API Reference]([presence_penalty](https://help.aliyun.com/zh/model-studio/developer-reference/use-qwen-by-calling-api))，来了解更多参数的使用说明。

4. 调优大模型：如果上述方法都做了充分的尝试，仍然不及预期，或者希望有更进一步的效果提升，你也可以尝试面向你的场景调优一个模型。在后续的章节中，我们将一起学习和实践这一点。
      

## ✅ 本节小结

通过前面的学习，你已经了解了一个简单 RAG 的工作流程，以及常见优化手段。你也可以结合前面学习到的知识，结合实际需求，将不同的问题，路由到不同的 RAG 应用中，以构建一个能力更强大的模块化 RAG 应用。当然，RAG 的优化手段远不止课程中介绍的这些，业内关于 RAG 的研究和探索也在持续进行，建议你持续保持了解和学习。

另外，从前面的学习可以看到，构建一个完善、表现足够好的 RAG 应用并不简单。

而在实际工作中，你可能需要更快的捕捉业务机会，没有时间投入到这些细节完善中。借助百炼，你可以快速构建一个效果比较好的 RAG 应用，而不去关注这么多细节。

- 比如，你可以参考这篇文档来[0代码构建私有知识问答应用](https://help.aliyun.com/zh/model-studio/getting-started/build-knowledge-base-qa-assistant-without-coding)，使用默认配置即可获得一个效果不错的 RAG 应用。

- 如果你的业务流程比较复杂，也可以借助百炼上的[可视化工作流、智能体编排](https://help.aliyun.com/zh/model-studio/user-guide/application-introduction)，来构建一个更强大的应用。
  
- 同时，百炼也提供了一系列 [LlamaIndex 组件](https://help.aliyun.com/zh/model-studio/developer-reference/llamaindex/)，方便你充分利用百炼的能力的同时，可以继续使用熟悉的 LlamaIndex API 构建 RAG 应用。


此外，通过前面的学习，你应该也能发现，大模型不只是可以用于构建问答系统。借助大模型识别用户意图、提取结构化信息（比如前面的根据用户问题提取标签），也能在很多其他应用场景中发挥作用。

## 🔥 课后小测验


【单选题】在RAG应用中，文档切片的长度和内容对检索效果有很大影响。如果文档切片长度过大，导致引入过多的干扰信息，应该如何处理？

A. 增加文档的数量

B. 减少切片长度，或根据业务特点开发更合理的切片策略

C. 使用更高级的检索算法

D. 提升大模型的训练水平

答案：B

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。